**Homework 1**

The purpose of Homework 1 is to use Python to perform data analysis to solve some business questions related to a hotels business.<br>
<br>


Some useful libraries - more could be necessary

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
from functools import reduce

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # ignore SettingWithCopyWarning

In [ ]:
# constants

MONTH_ORDER = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]
EQUATOR_LATITUDE = 0.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Question 0: Import the hotels data into your environment, and generate a unique ID.



In [ ]:
# update this to your filepath
hotels_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MSAI339: DS Seminar/hotel_data.csv")

In [ ]:
# unique IDs should start from 1
hotels_df.reset_index(inplace=True)
hotels_df.rename(columns={"index": "ID"}, inplace=True)
hotels_df['ID'] = hotels_df['ID'] + 1

In [ ]:
hotels_df.sample(5).T

Question 1: Create a function that displays the data type, number and percentage of null values, and number of unique values for each column of a given dataframe. (5 points)

In [ ]:
def analyze_dataframe(df):
    # fetching the data types
    data_types = hotels_df.dtypes.reset_index()
    data_types.columns = ["column_name", "data_type"]
    # count number of null values
    num_null_values = df.isna().sum().reset_index()
    num_null_values.columns = ["column_name", "num_null_values"]
    total_rows = df.shape[0]
    unique_values = df.nunique().reset_index()
    unique_values.columns = ["column_name", "num_unique_values"]
    # merging all dataframes
    description_df = reduce(
        lambda left, right: pd.merge(left, right, on=["column_name"], how="outer"),
        [data_types, num_null_values, unique_values],
    )
    description_df["perc_null_values"] = (
        description_df["num_null_values"] * 100.0 / total_rows
    )
    # number of column_names should match with df
    assert description_df.shape[0] == df.shape[1]

    display(description_df)

In [ ]:
analyze_dataframe(hotels_df)

Question 2A: Find the total number of visitors who have canceled a reservation. (1.5 points)

In [ ]:
is_canceled_mask = hotels_df["is_canceled"] == 1
print(
    f"""Total bookings that were canceled: {hotels_df.loc[is_canceled_mask, "ID"].count()}"""
)
print(
    f"""Total adults that canceled: {hotels_df.loc[is_canceled_mask, "adults"].sum()}"""
)
print(
    f"""Total people that canceled: {int(hotels_df.loc[is_canceled_mask, ["adults", "children", "babies"]].sum().sum())}"""
)

Question 2B: Which month has the most cancellations? (1.5 points)

Should we use `arrival_date_month` or `reservation_status_date` as the month for cancellations?

First, we'll check if there arrival date months for cancellations.

In [ ]:
hotels_df.loc[is_canceled_mask, ["arrival_date_month"]].value_counts()

Yes, there are arrival date months for cancellations. However, is the `reservation_status_date` different from the `arrival_date_month` for canceled customers?

In [ ]:
hotels_df["reservation_status_date"] = pd.to_datetime(
    hotels_df["reservation_status_date"]
)
hotels_df["reservation_status_month"] = hotels_df[
    "reservation_status_date"
].dt.strftime("%B")
hotels_canceled_months_df = hotels_df.loc[
    is_canceled_mask, ["arrival_date_month", "reservation_status_month", "ID"]
]
# do arrival and reservation months match?
hotels_canceled_months_df["is_arrival_reservation_month_match"] = (
    hotels_canceled_months_df["arrival_date_month"]
    == hotels_canceled_months_df["reservation_status_month"]
)
hotels_canceled_months_df["is_arrival_reservation_month_match"].value_counts(
    normalize=True
) * 100.0

Approximately 80% of the months do not match among arrival and reservation months for cancellation. Furthermore, the reservation status date is most likely the date at which the reservation was updated to cancelled. Hence, we'll use the reservation status month as the month a visit is canceled.

In [ ]:
# number of IDs that were canceled based on reservation status month
hotels_reservation_cancellations_df = (
    hotels_df.loc[is_canceled_mask]
    .groupby(["reservation_status_month"])["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "num_cancellations"})
)
most_cancellation_month = hotels_reservation_cancellations_df.loc[
    hotels_reservation_cancellations_df["num_cancellations"].idxmax(), :
]
print(
    f"""Month with most cancellations is {most_cancellation_month['reservation_status_month']} """
    + f"""with {most_cancellation_month['num_cancellations']} cancellations"""
)

Question 2C: Plot the number of cancellations per month in a histogram. (2 points)

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.barplot(
    x="reservation_status_month",
    y="num_cancellations",
    data=hotels_reservation_cancellations_df,
    order=MONTH_ORDER,
    color="Blue",
)
# Add data labels above each bar
for p in ax.patches:
    ax.annotate(
        f"{p.get_height():.0f}",
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="center",
        fontsize=9,
        color="grey",
        xytext=(0, 5),
        textcoords="offset points",
    )
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.xlabel("Month")
plt.ylabel("Number of Cancellations")
plt.title("Monthly Hotel Cancellations (based on reservations)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Question 3: Create a function that prints the percentage of canceled reservations for EACH hotel. (5 points)

In [ ]:
def hotel_cancellations(
    input_hotel_df,
    id_col="ID",
    hotel_col="hotel",
    cancellation_col="is_canceled",
):
    # count number of bookings per hotel
    num_bookings_per_hotel_df = (
        input_hotel_df.groupby([hotel_col])[id_col]
        .nunique()
        .reset_index()
        .rename(columns={id_col: "total_reservations"})
    )
    # count number of canceled reservation
    num_cancellations_df = (
        input_hotel_df[hotels_df[cancellation_col] == 1]
        .groupby([hotel_col])[id_col]
        .nunique()
        .reset_index()
        .rename(columns={id_col: "num_canceled_reservations"})
    )
    # calculating percentage
    num_cancellations_df = pd.merge(
        num_cancellations_df, num_bookings_per_hotel_df, on=hotel_col, how="inner"
    )
    num_cancellations_df["perc_cancellations"] = (
        num_cancellations_df["num_canceled_reservations"]
        * 100.0
        / num_cancellations_df["total_reservations"]
    )
    display(num_cancellations_df)

In [ ]:
hotel_cancellations(hotels_df)

Question 4: Filter the dataset so that it only contains visitors who have not canceled their reservations. Store this dataset as a separate variable (we will be using it going forward). (5 points)

In [ ]:
non_canceled_reservations_df = hotels_df[~is_canceled_mask]
print(non_canceled_reservations_df.shape)
display(non_canceled_reservations_df.sample(5).T)

Question 5A: Using the filtered dataset from Question 4, create a plot for each hotel showing the number of reservations for each month of arrival. Draw a conclusion on whether the trend is the same between the two hotels (in a sentence or two). (2.5 points)

In [ ]:
non_canceled_reservations_by_hotel_month_df = (
    non_canceled_reservations_df.groupby(["hotel", "arrival_date_month"])["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "num_reservations"})
)

In [ ]:
plt.figure(figsize=(14, 6))
ax = sns.barplot(
    x="arrival_date_month",
    y="num_reservations",
    hue="hotel",
    data=non_canceled_reservations_by_hotel_month_df,
    order=MONTH_ORDER,
)
# adding a label on each bar
for p in ax.patches:
    ax.annotate(
        f"{p.get_height():.0f}",
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="center",
        fontsize=9,
        color="grey",
        xytext=(0, 5),
        textcoords="offset points",
    )
plt.xlabel("Arrival Month")
plt.ylabel("Number of Reservations")
plt.title("Monthly Reservations by Hotel")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
ax.yaxis.grid(True, linestyle="--", alpha=0.7)
plt.show()

The trend looks fairly similar with the exception of May and December - Both hotels see an increasing number of reservations through January to March. The trend plateaus and drops a bit in April. However, the bookings increase in May for "City Hotel" (left) compared to "Resort Hotel" (right). The trend again gradually increases for both hotels and peaks in August. Finally it sees a steep decline until December.

However, let's plot it as a proportion of reservation and see if both hotels share the same trend. Visualizing proportions are far easier than absolute numbers - we can plot the two hotels on the same graph as proportions are normalized.

In [ ]:
# calculating proportions of reservations
# as number of reservation per month / total reservation
non_canceled_reservations_by_hotel_df = (
    non_canceled_reservations_df.groupby(["hotel"])["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "total_reservations"})
)
non_canceled_reservations_by_hotel_month_df = pd.merge(
    non_canceled_reservations_by_hotel_month_df,
    non_canceled_reservations_by_hotel_df,
    on="hotel",
    how="inner",
)
non_canceled_reservations_by_hotel_month_df["perc_reservations"] = (
    non_canceled_reservations_by_hotel_month_df["num_reservations"]
    * 100.0
    / non_canceled_reservations_by_hotel_month_df["total_reservations"]
)

In [ ]:
plt.figure(figsize=(14, 6))
ax = sns.barplot(
    x="arrival_date_month",
    y="perc_reservations",
    hue="hotel",
    data=non_canceled_reservations_by_hotel_month_df,
    order=MONTH_ORDER,
)
# adding a label on each bar
for p in ax.patches:
    ax.annotate(
        f"{p.get_height():.2f}",
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="center",
        fontsize=9,
        color="grey",
        xytext=(0, 5),
        textcoords="offset points",
    )
plt.xlabel("Arrival Month")
plt.ylabel("Proportion of Reservations")
plt.title("Proportion of Reservations by Hotel and Arrival Month")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
ax.yaxis.grid(True, linestyle="--", alpha=0.7)
plt.show()

**Conclusion**:

It is clear that the trends between both hotels look similar accross months with the exception of May and December. Both hotels see an increasing number of reservations through January to March, and drops in April. The general trend again gradually increases for both hotels and peaks in August, followed by a steep decline until December.


_Why?_

The trend seems to be representative of holidays in the summer months - July and August are the highest proportion of holidays. Winter month might be lower in proportion possible because people tend to travel more often during longer school break.

Question 5B: Using the filtered dataset from Question 4, create a plot for each hotel showing the number of reservations for each month of arrival, delineated by whether or not there are children (or babies) present on the reservation. (2.5 points)

In [ ]:
non_canceled_reservations_df.loc[:, "is_children_present"] = False
non_canceled_reservations_df.loc[
    (non_canceled_reservations_df[["children", "babies"]].sum(axis=1) > 0),
    "is_children_present",
] = True
non_canceled_reservations_by_hotel_month_children_df = (
    non_canceled_reservations_df.groupby(
        ["hotel", "arrival_date_month", "is_children_present"]
    )["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "num_reservations"})
)
non_canceled_reservations_by_hotel_month_children_df[
    "is_children_present"
] = non_canceled_reservations_by_hotel_month_children_df["is_children_present"].astype(
    str
)

In [ ]:
g = sns.FacetGrid(
    data=non_canceled_reservations_by_hotel_month_children_df,
    row="hotel",
    margin_titles=False,
    height=5,
    aspect=1.5,
    sharex=False,
)
g.map_dataframe(
    sns.barplot,
    x="arrival_date_month",
    y="num_reservations",
    hue="is_children_present",
    order=MONTH_ORDER,
    palette=sns.color_palette("Set1"),
)
g.set_axis_labels("Arrival Month", "Number of Reservations")
# Add a legend and a gridline
for ax in g.axes.flat:
    ax.legend(title="Children Present", loc="upper right")
    ax.yaxis.grid(True, linestyle="--", alpha=0.7)
# add hotel titles individual plot titles
hotel_titles = non_canceled_reservations_by_hotel_month_children_df["hotel"].unique()
for i, (ax, title) in enumerate(
    zip(
        g.axes.flat,
        hotel_titles,
    )
):
    ax.set_title(title)
g.set_xticklabels(rotation=45)
plt.tight_layout()
plt.show()

Question 6: Using the filtered dataset from Question 4, create a single column for "arrival_date" that shows the date of arrival in Year-Month-Day format. (5 points)

In [ ]:
non_canceled_reservations_df["arrival_date"] = pd.to_datetime(
    (
        non_canceled_reservations_df["arrival_date_year"].astype(str)
        + "-"
        + non_canceled_reservations_df["arrival_date_month"]
        + "-"
        + non_canceled_reservations_df["arrival_date_day_of_month"].astype(str)
    ),
    format="%Y-%B-%d",
)
display(
    non_canceled_reservations_df[
        [
            "arrival_date_year",
            "arrival_date_month",
            "arrival_date_day_of_month",
            "arrival_date",
        ]
    ].sample(5)
)
print(
    f"""DataType of `arrival_date` is {non_canceled_reservations_df["arrival_date"].dtype}."""
)

Question 7A: Create a new dataframe that contains the date of arrival and the number of reservations arriving on that particular day. Sort that dataframe by number of daily reservations, from highest to lowest. Display the first 20 rows of the dataframe. (2.5 points)

In [ ]:
non_canceled_reservations_per_arrival_day_df = (
    non_canceled_reservations_df.groupby(["arrival_date"])["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "num_reservations"})
    .sort_values(by=["num_reservations", "arrival_date"], ascending=[False, True])
)
top_20_non_canceled_reservations_per_arrival_day_df = non_canceled_reservations_per_arrival_day_df.head(20)
top_20_non_canceled_reservations_per_arrival_day_df['rank'] = top_20_non_canceled_reservations_per_arrival_day_df['num_reservations']\
.rank(ascending=False, method="dense")
top_20_non_canceled_reservations_per_arrival_day_df

Question 7B: Visualize the data from Question 7A as a line plot. (2.5 points)

In [ ]:
arrival_date_range = pd.date_range(
    start=non_canceled_reservations_per_arrival_day_df["arrival_date"].min(),
    end=non_canceled_reservations_per_arrival_day_df["arrival_date"].max(),
)
full_arrival_date_range_df = pd.DataFrame(
    {"arrival_date": arrival_date_range}
)
non_canceled_reservations_per_arrival_day_imputed_df = pd.merge(
    full_arrival_date_range_df,
    non_canceled_reservations_per_arrival_day_df,
    on="arrival_date",
    how="left",
).fillna({"num_reservations": 0})

In [ ]:
plt.figure(figsize=(15, 6.5))
sns.lineplot(
    data=non_canceled_reservations_per_arrival_day_imputed_df,
    x="arrival_date",
    y="num_reservations",
)
sns.scatterplot(
    data=top_20_non_canceled_reservations_per_arrival_day_df,
    x="arrival_date",
    y="num_reservations",
    label="Top 20 Reservations",
    color='orange',
)
plt.grid(True)
plt.title("Number of Reservations Over Arrival Date")
plt.xlabel("Arrival Date")
plt.ylabel("Number of Reservations")
# displaying the date once every three weeks
triweekly_xticks = pd.date_range(
    start=non_canceled_reservations_per_arrival_day_imputed_df["arrival_date"].min(),
    end=non_canceled_reservations_per_arrival_day_imputed_df["arrival_date"].max(),
    freq="3W-MON",
)
# marking the top 20 reservation
for _, row in top_20_non_canceled_reservations_per_arrival_day_df.iterrows():
    plt.text(
        row['arrival_date'],
        row['num_reservations'],
        f"#{int(row['rank'])}",
        ha='center',
        va='bottom'
    )
plt.xticks(rotation=45, ticks=triweekly_xticks)
plt.show()

In [ ]:
top_20_non_canceled_reservations_copy_df = top_20_non_canceled_reservations_per_arrival_day_df.copy()
top_20_non_canceled_reservations_copy_df['arrival_date'] = top_20_non_canceled_reservations_copy_df['arrival_date'].astype(str)

In [ ]:
plt.figure(figsize=(10, 6.5))
sns.lineplot(
    data=top_20_non_canceled_reservations_copy_df,
    x="arrival_date",
    y="num_reservations",
)
plt.grid(True)
plt.title("Top 20: Number of Reservations Over Arrival Date")
plt.xlabel("Number of Reservations")
plt.ylabel("Arrival Date")
plt.xticks(rotation=45)
plt.show()

Question 8A: What is the average daily reservation rate by hotel and customer type? Print your results in a dataframe and display all rows. (2.5 points)

Since `reservation_status_date` is the last date a reservation changes, we'll be ignoring canceled reservations. This is because cancelled reservations would contain the date the reservation was cancelled and not when the booking was made.

In [ ]:
non_canceled_reservations_total_df = (
    non_canceled_reservations_df.groupby(
        ["hotel", "customer_type"]
    )["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "num_reservations"})
)
# calculate this as number of bookings over a date range
arrival_date_range = (
    non_canceled_reservations_df["arrival_date"].max()
    - non_canceled_reservations_df["arrival_date"].min()
  ).total_seconds() / (60 * 60 * 24) + 1
non_canceled_reservations_total_df['daily_avg_reservations'] = (
  non_canceled_reservations_total_df['num_reservations'] / arrival_date_range
)
non_canceled_reservations_total_df

Question 8B: Plot the dataframe from Question 8A as a boxplot. (2.5 points)

In [ ]:
non_canceled_reservations_per_date_df = (
    non_canceled_reservations_df.groupby(
        ["hotel", "customer_type", "arrival_date"]
      )["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "num_reservations"})
)

In [ ]:
plt.figure(figsize=(12, 7))
customer_type_order = non_canceled_reservations_per_date_df.customer_type.unique()
ax = sns.boxplot(
    data=non_canceled_reservations_per_date_df,
    y="hotel",
    x="num_reservations",
    hue="customer_type",
    hue_order=customer_type_order,
)
# adding the mean to boxplot
stripplot = sns.stripplot(
    data=non_canceled_reservations_total_df,
    y="hotel",
    x="daily_avg_reservations",
    hue="customer_type",
    hue_order=customer_type_order,
    dodge=True,
    size=8,
    ax=ax,
)
handles, labels = ax.get_legend_handles_labels()
l = plt.legend(
    handles,
    labels[0:4] + [f"{h} Mean" for h in labels[4:]],
    bbox_to_anchor=(1.05, 1),
    loc=2,
    borderaxespad=0.0,
)
plt.grid(True)
plt.title("Distribution of Number of Daily Reservations by Hotel and Customer Type")
plt.ylabel("Hotel")
plt.xlabel("Number of Daily Reservations")
plt.show()

Question 9A: Import the "Country Codes and Coordinates" spreadsheet into your environment, and merge it with your filtered dataset from Question 4. Display the top 10 countries by number of reservations. (2.5 points)

In [ ]:
country_codes_coords_df = pd.read_csv(
    "/content/drive/My Drive/Colab Notebooks/MSAI339: DS Seminar/country_codes_and_coordinates - countries_codes_and_coordinates.csv"
)
country_codes_coords_df = country_codes_coords_df.drop_duplicates(
    [
        "Alpha-2 code",
        "Alpha-3 code",
        "Numeric code",
        "Latitude (average)",
        "Longitude (average)",
    ],
    keep="first",
).rename(
    columns={"Country": "Country Name"},
)
# check for duplicates
country_codes_coords_df.nunique(dropna=False)

Sanity Check - The number of `Alpha-2 code`, `Alpha-3 code` and `Country Name` should be the same.

In [ ]:
# fill missing countries
non_canceled_reservations_df["country"].fillna("Missing", inplace=True)
# merging with Alpha-3
non_canceled_reservations_country_info_v1_df = pd.merge(
    non_canceled_reservations_df,
    country_codes_coords_df[country_codes_coords_df["Alpha-3 code"].notnull()],
    left_on=["country"],
    right_on=["Alpha-3 code"],
    how="left",
)
# merging with Alpha-2 for those missing from Alpha-3
non_canceled_reservations_country_info_v2_df = pd.concat(
    [
        non_canceled_reservations_country_info_v1_df[
            non_canceled_reservations_country_info_v1_df["Country Name"].notnull()
        ],
        pd.merge(
            non_canceled_reservations_country_info_v1_df[
                non_canceled_reservations_country_info_v1_df["Country Name"].isna()
            ].drop(country_codes_coords_df.columns, axis=1),
            country_codes_coords_df[country_codes_coords_df["Alpha-2 code"].notnull()],
            left_on=["country"],
            right_on=["Alpha-2 code"],
            how="left",
        ),
    ]
)

# check if we lost any data in the merge
assert (
    non_canceled_reservations_country_info_v2_df.shape[0]
    == non_canceled_reservations_df.shape[0]
)
non_canceled_reservations_country_info_v2_df["Country Name"].fillna(
    "Missing", inplace=True
)

# printing the top 10 countries
reservations_by_country_df = (
    non_canceled_reservations_country_info_v2_df.groupby(["Country Name"])["ID"]
    .nunique()
    .reset_index()
    .rename(columns={"ID": "num_reservations"})
)
reservations_by_country_df["num_reservation_rank"] = reservations_by_country_df[
    "num_reservations"
].rank(ascending=False, method="dense")
top_10_reservations_by_country_df = reservations_by_country_df[
    reservations_by_country_df["num_reservation_rank"] <= 10
].sort_values(by=["num_reservation_rank"])
top_10_reservations_by_country_df

Question 9B: Visualize the data from Question 9A as a horizontal bar plot. (2.5 points)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    x="num_reservations",
    y="Country Name",
    data=top_10_reservations_by_country_df,
    palette="viridis",
)
plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.xlabel("Number of Reservations")
plt.ylabel("Country")
plt.title("Top 10 Reservations by Country")
plt.show()

Question 10A: How many guests (including children and babies) are present on the average reservation? (2.5 points)

Option 1: For all reservations.

In [ ]:
hotels_df["total_personnel"] = hotels_df[["adults", "children", "babies"]].sum(axis=1)
print(f"""Average guests per reservation: {hotels_df["total_personnel"].mean():.2f}""")

Option 2: For only non-canceled reservations.

In [ ]:
non_canceled_reservations_country_info_v2_df[
    "total_personnel"
] = non_canceled_reservations_country_info_v2_df[["adults", "children", "babies"]].sum(
    axis=1
)
print(
    f"""Average guests per reservation: {non_canceled_reservations_country_info_v2_df["total_personnel"].mean():.2f}"""
)

Question 10B: True or False - Hotels below the equator have a higher average number of guests on reservations. (2.5 points)

Ignoring reservations with missing countries as we don't know what their coordinates are.

Option 1: For all reservations.

In [ ]:
# merging with Alpha-3 first
hotels_country_info_v1_df = pd.merge(
    hotels_df[hotels_df["country"].notnull()],
    country_codes_coords_df[country_codes_coords_df["Alpha-3 code"].notnull()],
    left_on=["country"],
    right_on=["Alpha-3 code"],
    how="left",
)
# merging the missing ones with Alpha-2
hotels_country_info_v2_df = pd.concat(
    [
        hotels_country_info_v1_df[hotels_country_info_v1_df["Country Name"].notnull()],
        pd.merge(
            hotels_country_info_v1_df[
                hotels_country_info_v1_df["Country Name"].isna()
            ].drop(country_codes_coords_df.columns, axis=1),
            country_codes_coords_df[country_codes_coords_df["Alpha-2 code"].notnull()],
            left_on=["country"],
            right_on=["Alpha-2 code"],
            how="left",
        ),
    ]
)

# check if we lost any data in the merge
assert (
    hotels_country_info_v2_df.shape[0]
    == hotels_df[hotels_df["country"].notnull()].shape[0]
)
# is country above or below the equator?
hotels_country_info_v2_df["Equator_Category"] = hotels_country_info_v1_df[
    "Latitude (average)"
].apply(lambda x: "Above Equator" if x > EQUATOR_LATITUDE else "Below Equator")
avg_guests_per_resv_by_equator_df = (
    hotels_country_info_v2_df.groupby(["Equator_Category"])["total_personnel"]
    .mean()
    .reset_index()
    .rename(columns={"total_personnel": "average_guests_per_reservation"})
)

In [ ]:
print(
    "Hotels below the equator have a higher average number of guests on reservations: "
    + str(
        avg_guests_per_resv_by_equator_df.loc[
            avg_guests_per_resv_by_equator_df["Equator_Category"] == "Below Equator",
            "average_guests_per_reservation",
        ].item()
        > avg_guests_per_resv_by_equator_df.loc[
            avg_guests_per_resv_by_equator_df["Equator_Category"] == "Above Equator",
            "average_guests_per_reservation",
        ].item()
    )
)
display(avg_guests_per_resv_by_equator_df)

Option 2: For only non-canceled reservations.

In [ ]:
non_missing_non_canceled_reservations_country_info_v2_df = (
    non_canceled_reservations_country_info_v2_df[
        non_canceled_reservations_country_info_v2_df["Country Name"] != "Missing"
    ]
)
# is country above or below the equator?
non_missing_non_canceled_reservations_country_info_v2_df[
    "Equator_Category"
] = non_missing_non_canceled_reservations_country_info_v2_df[
    "Latitude (average)"
].apply(
    lambda x: "Above Equator" if x > EQUATOR_LATITUDE else "Below Equator"
)
avg_guests_per_resv_by_equator_df = (
    non_missing_non_canceled_reservations_country_info_v2_df.groupby(
        ["Equator_Category"]
    )["total_personnel"]
    .mean()
    .reset_index()
    .rename(columns={"total_personnel": "average_guests_per_reservation"})
)

In [ ]:
print(
    "Hotels below the equator have a higher average number of guests on reservations: "
    + str(
        avg_guests_per_resv_by_equator_df.loc[
            avg_guests_per_resv_by_equator_df["Equator_Category"] == "Below Equator",
            "average_guests_per_reservation",
        ].item()
        > avg_guests_per_resv_by_equator_df.loc[
            avg_guests_per_resv_by_equator_df["Equator_Category"] == "Above Equator",
            "average_guests_per_reservation",
        ].item()
    )
)
display(avg_guests_per_resv_by_equator_df)

Given both options, "Hotels below the equator have a higher average number of guests on reservations" is True.